# Training a model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset/Draft/filtered_data_50k-users.csv')
df = df.drop(columns=['Unnamed: 0'])
# sampledf = df['user_id'].sample(n=50000)
# df = df[df['user_id'].isin(list(sampledf))]
df = df[['user_id', 'movie_id', 'rating']]
# df['user_id'] = df['user_id'].rank(method='dense')
# df['movie_id'] = df['movie_id'].rank(method='dense')

# from pathlib import Path  
# filepath = Path('/content/drive/MyDrive/CS412_project/Dataset/Draft/filtered_data_50k-users.csv')  
# df.to_csv(filepath)

# df = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset//Draft/filtered_data_50k-users.csv')
df.head()

,user_id,movie_id,rating
0,1488844,1,3.000
1,822109,1,5.000
2,30878,1,4.000
3,1711859,1,4.000
4,558634,1,4.000


In [ ]:
# df = df.drop(columns=['Unnamed: 0'])
# df['user_id'] = df['user_id'].rank(method='dense')
# df['movie_id'] = df['movie_id'].rank(method='dense')


In [ ]:
from __future__ import print_function

import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols
USER_RATINGS = False
# !pip install --upgrade -q gspread
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

In [ ]:
def build_rating_sparse_tensor(ratings_df):
  """
  Args:
    ratings_df: a pd.DataFrame with `user_id`, `movie_id` and `rating` columns.
  Returns:
    a tf.SparseTensor representing the ratings matrix.
  """
  indices = ratings_df[['user_id', 'movie_id']].values
  values = ratings_df['rating'].values
  print(ratings_df['user_id'].nunique())
  print(ratings_df['movie_id'].nunique())
  # return tf.SparseTensor(
  #     indices=indices,
  #     values=values,
  #     dense_shape=[df['user_id'].nunique(), df['movie_id'].nunique()])
  return tf.SparseTensor(
      indices=indices,
      values=values,
      dense_shape=[df['user_id'].max(), df['movie_id'].max()])

In [ ]:
def sparse_mean_square_error(sparse_ratings, user_embeddings, movie_embeddings):
  """
  Args:
    sparse_ratings: A SparseTensor rating matrix, of dense_shape [N, M]
    user_embeddings: A dense Tensor U of shape [N, k] where k is the embedding
      dimension, such that U_i is the embedding of user i.
    movie_embeddings: A dense Tensor V of shape [M, k] where k is the embedding
      dimension, such that V_j is the embedding of movie j.
  Returns:
    A scalar Tensor representing the MSE between the true ratings and the
      model's predictions.
  """
  predictions = tf.gather_nd(
      tf.matmul(user_embeddings, movie_embeddings, transpose_b=True),
      sparse_ratings.indices)
  loss = tf.losses.mean_squared_error(sparse_ratings.values, predictions)
  return loss

In [ ]:
class CFModel(object):
  """Simple class that represents a collaborative filtering model"""
  def __init__(self, embedding_vars, loss, metrics=None):
    """Initializes a CFModel.
    Args:
      embedding_vars: A dictionary of tf.Variables.
      loss: A float Tensor. The loss to optimize.
      metrics: optional list of dictionaries of Tensors. The metrics in each
        dictionary will be plotted in a separate figure during training.
    """
    self._embedding_vars = embedding_vars
    self._loss = loss
    self._metrics = metrics
    self._embeddings = {k: None for k in embedding_vars}
    self._session = None

  @property
  def embeddings(self):
    """The embeddings dictionary."""
    return self._embeddings

  def train(self, num_iterations=100, learning_rate=1.0, plot_results=True,
            optimizer=tf.train.GradientDescentOptimizer):
    """Trains the model.
    Args:
      iterations: number of iterations to run.
      learning_rate: optimizer learning rate.
      plot_results: whether to plot the results at the end of training.
      optimizer: the optimizer to use. Default to GradientDescentOptimizer.
    Returns:
      The metrics dictionary evaluated at the last iteration.
    """
    with self._loss.graph.as_default():
      opt = optimizer(learning_rate)
      train_op = opt.minimize(self._loss)
      local_init_op = tf.group(
          tf.variables_initializer(opt.variables()),
          tf.local_variables_initializer())
      if self._session is None:
        self._session = tf.Session()
        with self._session.as_default():
          self._session.run(tf.global_variables_initializer())
          self._session.run(tf.tables_initializer())
          tf.train.start_queue_runners()

    with self._session.as_default():
      local_init_op.run()
      iterations = []
      metrics = self._metrics or ({},)
      metrics_vals = [collections.defaultdict(list) for _ in self._metrics]

      # Train and append results.
      for i in range(num_iterations + 1):
        _, results = self._session.run((train_op, metrics))
        if (i % 10 == 0) or i == num_iterations:
          print("\r iteration %d: " % i + ", ".join(
                ["%s=%f" % (k, v) for r in results for k, v in r.items()]),
                end='')
          iterations.append(i)
          for metric_val, result in zip(metrics_vals, results):
            for k, v in result.items():
              metric_val[k].append(v)

      for k, v in self._embedding_vars.items():
        self._embeddings[k] = v.eval()

      if plot_results:
        # Plot the metrics.
        num_subplots = len(metrics)+1
        fig = plt.figure()
        fig.set_size_inches(num_subplots*10, 8)
        for i, metric_vals in enumerate(metrics_vals):
          ax = fig.add_subplot(1, num_subplots, i+1)
          for k, v in metric_vals.items():
            ax.plot(iterations, v, label=k)
          ax.set_xlim([1, num_iterations])
          ax.legend()
      return results

In [ ]:
def build_model(ratings, embedding_dim=3, init_stddev=1.):
  """
  Args:
    ratings: a DataFrame of the ratings
    embedding_dim: the dimension of the embedding vectors.
    init_stddev: float, the standard deviation of the random initial embeddings.
  Returns:
    model: a CFModel.
  """
  # Split the ratings DataFrame into train and test.
  train_ratings, test_ratings = split_dataframe(ratings)
  # SparseTensor representation of the train and test datasets.
  A_train = build_rating_sparse_tensor(train_ratings)
  A_test = build_rating_sparse_tensor(test_ratings)
  # Initialize the embeddings using a normal distribution.
  U = tf.Variable(tf.random_normal(
      [A_train.dense_shape[0], embedding_dim], stddev=init_stddev))
  V = tf.Variable(tf.random_normal(
      [A_train.dense_shape[1], embedding_dim], stddev=init_stddev))
  train_loss = sparse_mean_square_error(A_train, U, V)
  test_loss = sparse_mean_square_error(A_test, U, V)
  metrics = {
      'train_error': train_loss,
      'test_error': test_loss
  }
  embeddings = {
      "user_id": U,
      "movie_id": V
  }
  return CFModel(embeddings, train_loss, [metrics])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset/filtered_data_50k-users.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,user_id,movie_id,rating
0,24334.000,1.000,3.000
1,13478.000,1.000,3.000
2,17737.000,1.000,3.000
3,19363.000,1.000,3.000
4,20117.000,1.000,4.000


In [ ]:
df.head()

,user_id,movie_id,rating
0,1488844,1,3.000
1,822109,1,5.000
2,30878,1,4.000
3,1711859,1,4.000
4,558634,1,4.000


In [ ]:
# Build the CF model and train it.
model = build_model(df, embedding_dim=20, init_stddev=0.5)
model.train(num_iterations=1000, learning_rate=50.)


43053
13077
43049
13074


ResourceExhaustedError: ignored

In [ ]:
# model = build_model(df, embedding_dim=20, init_stddev=0.5)
# model.train(num_iterations=1000, learning_rate=40.)
{'train_error': 0.9869805, 'test_error': 1.0133865}

# model = build_model(df, embedding_dim=10, init_stddev=0.5)
# model.train(num_iterations=1000, learning_rate=40.)
{'train_error': 0.967465, 'test_error': 0.9813157}


# model = build_model(df, embedding_dim=50, init_stddev=0.5)
# model.train(num_iterations=1000, learning_rate=40.)
{'train_error': 1.0464747, 'test_error': 1.1178473}

# model = build_model(df, embedding_dim=80, init_stddev=0.5)
# model.train(num_iterations=1000, learning_rate=40.)
{'train_error': 1.0997438, 'test_error': 1.2243268}

# model = build_model(df, embedding_dim=100, init_stddev=0.5)
# model.train(num_iterations=1000, learning_rate=40.)
{'train_error': 1.137175, 'test_error': 1.309747}




AttributeError: ignored

In [ ]:
df_orig = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset//Draft/filtered_data_50k-users.csv')
df_orig.head()

,Unnamed: 0,user_id,movie_id,rating
0,0,1488844,1,3.000
1,1,822109,1,5.000
2,3,30878,1,4.000
3,18,1711859,1,4.000
4,22,558634,1,4.000


In [ ]:
df_orig['user_id_dense'] = df_orig['user_id'].rank(method='dense')
df_orig['movie_id_dense'] = df_orig['movie_id'].rank(method='dense')

In [ ]:
df_orig.head()

,Unnamed: 0,user_id,movie_id,rating,user_id_dense,movie_id_dense
0,0,1488844,1,3.000,24322.000,1.000
1,1,822109,1,5.000,13454.000,1.000
2,3,30878,1,4.000,504.000,1.000
3,18,1711859,1,4.000,27931.000,1.000
4,22,558634,1,4.000,9105.000,1.000


# Playing with embedding

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset/movie_titles.csv', names=['movie_id', 'year', 'title'],encoding='latin-1', on_bad_lines='skip')
movies.head()

,movie_id,year,title
0,1,2003.000,Dinosaur Planet
1,2,2004.000,Isle of Man TT 2004 Review
2,3,1997.000,Character
3,4,1994.000,Paula Abdul's Get Up & Dance
4,5,2004.000,The Rise and Fall of ECW


In [ ]:
new_row = pd.DataFrame({'movie_id':0, 'year':2022, 'title':"Empty"},
                                                            index =[0])
# simply concatenate both dataframes
movies = pd.concat([new_row, movies]).reset_index(drop = True)
movies.head()

,movie_id,year,title
0,0,2022.000,Empty
1,1,2003.000,Dinosaur Planet
2,2,2004.000,Isle of Man TT 2004 Review
3,3,1997.000,Character
4,4,1994.000,Paula Abdul's Get Up & Dance


In [ ]:
movie_embs = np.load('/content/drive/MyDrive/CS412_project/RecommendationModel/movie_embeddings_25perc_users.npy')

In [ ]:
movies[movies["title"].str.contains('Lion King')]

,movie_id,year,title
1682,1683,1994.000,The Lion King: Circle of Life: Sing-Along Songs
3078,3079,1994.000,The Lion King: Special Edition
8770,8771,2004.000,The Lion King 1 1/2: Bonus Material
10775,10776,1994.000,The Lion King: Special Edition: Bonus Material
16221,16222,2004.000,The Lion King 1 1/2
16659,16660,1998.000,The Lion King II: Simba's Pride


In [ ]:
DOT = 'dot'
COSINE = 'cosine'
def compute_scores(query_embedding, item_embeddings, measure=DOT):
  """Computes the scores of the candidates given a query.
  Args:
    query_embedding: a vector of shape [k], representing the query embedding.
    item_embeddings: a matrix of shape [N, k], such that row i is the embedding
      of item i.
    measure: a string specifying the similarity measure to be used. Can be
      either DOT or COSINE.
  Returns:
    scores: a vector of shape [N], such that scores[i] is the score of item i.
  """
  u = query_embedding
  V = item_embeddings
  if measure == COSINE:
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    u = u / np.linalg.norm(u)
  scores = u.dot(V.T)
  return scores

In [ ]:
def movie_neighbors(title_substring, measure=DOT, k=6):
  # Search for movie ids that match the given substring.
  #ids =  movies[movies['title'].str.contains(title_substring)].index.values
  ids =  movies[movies['title']==title_substring].index.values
  titles = movies.iloc[ids]['title'].values
  if len(titles) == 0:
    raise ValueError("Found no movies with title %s" % title_substring)
  print("Nearest neighbors of : %s." % titles[0])
  if len(titles) > 1:
    print("[Found more than one matching movie. Other candidates: {}]".format(
        ", ".join(titles[1:])))
  movie_id = ids[0]
  scores = compute_scores(
      movie_embs[movie_id], movie_embs,
      measure)
  score_key = measure + ' score'
  df = pd.DataFrame({
      score_key: list(scores),
      'titles': movies['title']
  })
  display.display(df.sort_values([score_key], ascending=False).head(k))

In [ ]:
movies[movies['title'].str.contains('Lord of')]

,movie_id,year,title,title_small
2412,2412,2004.000,Harry Potter and the Prisoner of Azkaban: Bonu...,harry potter and the prisoner of azkaban: bonu...
7114,7114,2002.000,Discovering the Real World of Harry Potter,discovering the real world of harry potter
11443,11443,2002.000,Harry Potter and the Chamber of Secrets,harry potter and the chamber of secrets
12338,12338,2004.000,Harry Potter and the Prisoner of Azkaban,harry potter and the prisoner of azkaban
13256,13256,2004.000,J.K. Rowling and the Birth of Harry Potter,j.k. rowling and the birth of harry potter
13315,13315,2001.000,Harry Potter and the Sorcerer's Stone: Bonus M...,harry potter and the sorcerer's stone: bonus m...
13351,13351,2002.000,Harry Potter and the Chamber of Secrets: Bonus...,harry potter and the chamber of secrets: bonus...
17627,17627,2001.000,Harry Potter and the Sorcerer's Stone,harry potter and the sorcerer's stone


In [ ]:
movie_neighbors("Harry Potter and the Prisoner of Azkaban", DOT)

Nearest neighbors of : Harry Potter and the Prisoner of Azkaban.


,dot score,titles
3456,17.381,Lost: Season 1
14961,17.178,Lord of the Rings: The Return of the King: Ext...
7230,17.139,The Lord of the Rings: The Fellowship of the R...
7833,17.128,Arrested Development: Season 2
7057,17.100,Lord of the Rings: The Two Towers: Extended Ed...
11662,16.965,The Sopranos: Season 5


In [ ]:
movie_neighbors("Harry Potter and the Prisoner of Azkaban", COSINE)

Nearest neighbors of : Harry Potter and the Prisoner of Azkaban.


,cosine score,titles
12338,1.000,Harry Potter and the Prisoner of Azkaban
3962,1.000,Finding Nemo (Widescreen)
2372,1.000,The Bourne Supremacy
11089,1.000,Monsters
14961,1.000,Lord of the Rings: The Return of the King: Ext...
14621,1.000,Shrek (Full-screen)


In [ ]:
movie_neighbors("Shrek (Full-screen)", COSINE)

Nearest neighbors of : Shrek (Full-screen).


,cosine score,titles
14621,1.000,Shrek (Full-screen)
3962,1.000,Finding Nemo (Widescreen)
3938,1.000,Shrek 2
2372,1.000,The Bourne Supremacy
12155,1.000,Spider-Man 2
11781,1.000,Indiana Jones and the Temple of Doom


In [ ]:
movie_neighbors("Carrie", COSINE)

Nearest neighbors of : Carrie.
[Found more than one matching movie. Other candidates: Carrie, Carrie]


,cosine score,titles
12890,1.000,Carrie
10583,1.000,The School of Rock
14999,1.000,Monster
2348,1.000,The Omen
9689,1.000,The Exorcist: Restored Version
10225,1.000,Groundhog Day


In [ ]:
genres = pd.read_csv('/content/drive/MyDrive/CS412_project/Dataset/netflix_titles.csv', encoding='latin-1', on_bad_lines='skip')
genres.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
movies['title_small']=movies['title'].str.lower()

In [ ]:
genres['title_small']=genres['title'].str.lower()

In [ ]:
new_df = pd.merge(movies, genres,  how='inner', left_on=['title_small','year'], right_on = ['title_small','release_year'])
new_df[new_df['listed_in'].str.contains('Documentaries')].head(20)


,movie_id,year,title_x,title_small,show_id,type,title_y,director,cast,country,date_added,release_year,rating,duration,listed_in,description
20,955,1997.000,Rhyme & Reason,rhyme & reason,s171,Movie,Rhyme & Reason,Peter Spirer,"Too $hort, B-Real, Kurtis Blow, Da Brat, Grand...",United States,"September 1, 2021",1997,R,89 min,"Documentaries, Music & Musicals",The world and culture of rap song topics such ...
30,1365,1998.000,Kurt & Courtney,kurt & courtney,s7245,Movie,Kurt & Courtney,Nick Broomfield,"Kurt Cobain, Courtney Love",United Kingdom,"January 8, 2019",1998,R,95 min,"Documentaries, Music & Musicals",This film charts Nirvana's Kurt Cobain's rise ...
49,2262,2002.000,Biggie & Tupac,biggie & tupac,s6310,Movie,Biggie & Tupac,Nick Broomfield,Nick Broomfield,United Kingdom,"January 26, 2019",2002,R,108 min,"Documentaries, Music & Musicals",British documentarian Nick Broomfield investig...
234,10449,2005.000,No Direction Home: Bob Dylan,no direction home: bob dylan,s2633,Movie,No Direction Home: Bob Dylan,Martin Scorsese,"Bob Dylan, Joan Baez, Dave Van Ronk, Peter Yar...","United Kingdom, United States, Japan","April 25, 2020",2005,TV-MA,208 min,"Documentaries, Music & Musicals",Featuring rare concert footage and interviews ...
279,12367,2005.000,Rize,rize,s1917,Movie,Rize,David LaChapelle,"Tommy the Clown, Tight Eyez","United States, United Kingdom","October 1, 2020",2005,PG-13,87 min,"Documentaries, Music & Musicals","This documentary explores ""krumping,"" a fast-p..."
284,12462,1973.000,Jimi Hendrix,jimi hendrix,s7131,Movie,Jimi Hendrix,Joe Boyd,"Jimi Hendrix, Eric Clapton, Billy Cox",United States,"November 1, 2019",1973,R,102 min,"Documentaries, Music & Musicals","Jimi Hendrix's family, friends, and fellow mus..."
293,12794,1942.000,The Battle of Midway,the battle of midway,s8206,Movie,The Battle of Midway,John Ford,"Henry Fonda, Jane Darwell",United States,"March 31, 2017",1942,TV-14,18 min,"Classic Movies, Documentaries",Director John Ford captures combat footage of ...
338,14681,1990.000,Paris Is Burning,paris is burning,s7709,Movie,Paris Is Burning,Jennie Livingston,NaN,United States,"February 1, 2017",1990,R,77 min,"Classic Movies, Cult Movies, Documentaries",This Sundance prize-winning documentary is an ...
